In [39]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# pd.set_option('display.max_columns', None)

path = "C:/Users/kleinow/ownCloud/!PROJECT EVOLGATH/Evolgath/updated_sql_search"

df names indicating the  queried keywords and where they had been found ( no addition = in citing paper's keywords, _ab = in citing paper's abstract, _titles = in citing paper's titles)

In [40]:
refs_emcoop = pd.read_csv(path + "/" + "refs_emcoop.csv")
refs_emcoop_ab = pd.read_csv(path + "/" + "refs_emcoop_ab.csv")
refs_emcoop_titles = pd.read_csv(path + "/" + "refs_emcoop_titles.csv")
refs_evolcoop = pd.read_csv(path + "/" + "refs_evolcoop.csv")
refs_evolcoop_ab = pd.read_csv(path + "/" + "refs_evolcoop_ab.csv")
refs_evolcoop_titles = pd.read_csv(path + "/" + "refs_evolcoop_titles.csv")
refs_evolgath = pd.read_csv(path + "/" + "refs_evolgath.csv")
refs_evolgath_ab = pd.read_csv(path + "/" + "refs_evolgath_ab.csv")
refs_evolgath_titles = pd.read_csv(path + "/" + "refs_evolgath_titles.csv")

C:\Users\kleinow\AppData\Local\Temp\ipykernel_22044\1595709130.py:7: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  refs_evolgath = pd.read_csv(path + "/" + "refs_evolgath.csv")


In [41]:
# append dfs according to the keywords
merged_evolgath = pd.concat([refs_evolgath, refs_evolgath_ab, refs_evolgath_titles], ignore_index=True)
merged_emcoop = pd.concat([refs_emcoop, refs_emcoop_ab, refs_emcoop_titles], ignore_index=True)
merged_evolcoop = pd.concat([refs_evolcoop, refs_evolcoop_ab, refs_evolcoop_titles], ignore_index=True)
merged_coop = pd. concat([merged_emcoop, merged_evolcoop], ignore_index=True)

# shape of each merged dataframe
print("Merged evolgath DataFrame shape:", merged_evolgath.shape)
print("Merged emcoop DataFrame shape:", merged_emcoop.shape)
print("Merged evolcoop DataFrame shape:", merged_evolcoop.shape)
print("Merged coop DataFrame shape:", merged_coop.shape)

# evolgath at this point still bigger than merged coop

Merged evolgath DataFrame shape: (142910, 16)
Merged emcoop DataFrame shape: (18820, 16)
Merged evolcoop DataFrame shape: (109114, 16)
Merged coop DataFrame shape: (127934, 16)


In [42]:
# add column with lower case titles
merged_evolgath['title_lower'] = merged_evolgath['ref_item_title'].str.lower()
merged_coop['title_lower'] = merged_coop['ref_item_title'].str.lower()   
merged_emcoop['title_lower'] = merged_emcoop['ref_item_title'].str.lower()
merged_evolcoop['title_lower'] = merged_evolcoop['ref_item_title'].str.lower()

len(set(merged_evolgath.title_lower) & set(merged_coop.title_lower)) # 8887 common titles before filtering
merged_evolgath.title_lower.nunique() # 53648, drop na as default
merged_coop.title_lower.nunique() # 28861

merged_evolgath.to_csv('merged_evolgath.csv', index=False)
merged_emcoop.to_csv('merged_emcoop.csv', index=False)
merged_evolcoop.to_csv('merged_evolcoop.csv', index=False)
merged_coop.to_csv('merged_coop.csv', index=False)

Try the first citation threshold: cited more than once, otherwise no connection to other papers possible

In [43]:
series_evolgath = merged_evolgath.groupby("item_id_cited")['item_id_citing'].nunique() #group the cited id by the citing id
#print(series_evolgath) #Length: 73306

series_coop = merged_coop.groupby("item_id_cited")['item_id_citing'].nunique()
#print(series_coop) # 42797

# for evolgath:
# cited items with more than one citing item
ab1_evol = series_evolgath.loc[series_evolgath.values > 1]

# show positions of the cited items with more than one citing item
merged_evolgath.loc[merged_evolgath.item_id_cited.isin(ab1_evol.index)] # 65285 rows

,item_id_citing,item_id_cited,citing_pubyear,ref_seq_nr,ref_pubyear,ref_item_title,ref_source_title,ref_authors,ref_volume,scopus_ref_issue,ref_pages,ref_doi,wos_ref_article_number,scopus_ref_text,scopus_ref_fulltext,wos_citation_context,title_lower
0,WOS:000508289100054,WOS:000223113500001,2017,1,2004.0,Toward a metabolic theory of ecology,ECOLOGY,"{""Brown, JH""}",85,NaN,1771,NaN,NaN,NaN,NaN,NaN,toward a metabolic theory of ecology
2,WOS:000508289100054,WOS:000073092100002,2017,3,1998.0,On economic applications of evolutionary game ...,JOURNAL OF EVOLUTIONARY ECONOMICS,"{""Friedman, D""}",8,NaN,15,NaN,NaN,NaN,NaN,NaN,on economic applications of evolutionary game ...
19,WOS:000274798200004,WOS:A1989U899000001,2009,5,1989.0,"EPIDEMIOLOGICAL MODELS WITH AGE STRUCTURE, PRO...",JOURNAL OF MATHEMATICAL BIOLOGY,"{""CASTILLOCHAVEZ, C""}",27,NaN,233,NaN,NaN,NaN,NaN,NaN,"epidemiological models with age structure, pro..."
21,WOS:000274798200004,WOS:000188753800045,2009,7,2004.0,Evolutionary dynamics of biological games,SCIENCE,"{""Nowak, MA""}",303,NaN,793,NaN,NaN,NaN,NaN,NaN,evolutionary dynamics of biological games
22,WOS:000274798200004,WOS:000223799100054,2009,8,2004.0,Vaccination and the theory of games,PROCEEDINGS OF THE NATIONAL ACADEMY OF SCIENCE...,"{""Bauch, CT""}",101,NaN,13391,NaN,NaN,NaN,NaN,NaN,vaccination and the theory of games
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142899,WOS:000076056600001,WOS:A1992JM57700006,1998,68,1992.0,EVOLUTIONARY STABILITY IN ASYMMETRIC GAMES,JOURNAL OF ECONOMIC THEORY,"{""SAMUELSON, L""}",57,NaN,363,NaN,NaN,NaN,NaN,NaN,evolutionary stability in asymmetric games
142902,WOS:000076056600001,WOS:A1989CC40500006,1998,71,1989.0,SPONTANEOUS ORDER,JOURNAL OF ECONOMIC PERSPECTIVES,"{""SUGDEN, R""}",3,NaN,85,NaN,NaN,NaN,NaN,NaN,spontaneous order
142903,WOS:000076056600001,WOS:A1993MD68700030,1998,72,1993.0,RECENT ADVANCES IN EVOLUTIONARY ECONOMICS - CO...,ECONOMICS LETTERS,"{""SAMUELSON, L""}",42,NaN,313,NaN,NaN,NaN,NaN,NaN,recent advances in evolutionary economics - co...
142905,WOS:000076056600001,WOS:A1993KH52900002,1998,74,1993.0,"LEARNING, MUTATION, AND LONG-RUN EQUILIBRIA IN...",ECONOMETRICA,"{""KANDORI, M""}",61,NaN,29,NaN,NaN,NaN,NaN,NaN,"learning, mutation, and long-run equilibria in..."


In [44]:
# for coop
ab1_coop = series_coop.loc[series_coop.values > 1]
merged_coop.loc[merged_coop.item_id_cited.isin(ab1_coop.index)] # 87708 rows

,item_id_citing,item_id_cited,citing_pubyear,ref_seq_nr,ref_pubyear,ref_item_title,ref_source_title,ref_authors,ref_volume,scopus_ref_issue,ref_pages,ref_doi,wos_ref_article_number,scopus_ref_text,scopus_ref_fulltext,wos_citation_context,title_lower
0,WOS:000803916300001,WOS:000188753800045,2022,8,2004.0,Evolutionary dynamics of biological games,SCIENCE,"{""Nowak, MA""}",303,NaN,793,NaN,NaN,NaN,NaN,NaN,evolutionary dynamics of biological games
1,WOS:000803916300001,WOS:000241921500011,2022,6,2006.0,Cooperation prevails when individuals adjust t...,PLOS COMPUTATIONAL BIOLOGY,"{""Santos, FC""}",2,NaN,1284,10.1371/journal.pcbi.0020140,NaN,NaN,NaN,NaN,cooperation prevails when individuals adjust t...
4,WOS:000803916300001,WOS:000301775800001,2022,2,2012.0,Human behavior in Prisoner's Dilemma experimen...,SCIENTIFIC REPORTS,"{""Gracia-Lazaro, C""}",2,NaN,ARTN 325,10.1038/srep00325,NaN,NaN,NaN,NaN,human behavior in prisoner's dilemma experimen...
5,WOS:000803916300001,WOS:000369085100062,2022,3,2016.0,Conditional cooperation and confusion in publi...,PROCEEDINGS OF THE NATIONAL ACADEMY OF SCIENCE...,"{""Burton-Chellew, MN""}",113,NaN,1291,10.1073/pnas.1509740113,NaN,NaN,NaN,NaN,conditional cooperation and confusion in publi...
6,WOS:000803916300001,WOS:000322521900002,2022,4,2013.0,Emergence of social networks via direct and in...,AUTONOMOUS AGENTS AND MULTI-AGENT SYSTEMS,"{""Phelps, S""}",27,NaN,355,10.1007/s10458-012-9207-8,NaN,NaN,NaN,NaN,emergence of social networks via direct and in...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127925,WOS:000423016900007,WOS:000232993500002,2012,4,2005.0,Appearances can be deceiving: Lessons learned ...,JASSS-THE JOURNAL OF ARTIFICIAL SOCIETIES AND ...,"{""Galan, JM""}",8,NaN,ARTN 2,NaN,NaN,NaN,NaN,NaN,appearances can be deceiving: lessons learned ...
127926,WOS:000423016900007,WOS:A1986F589100001,2012,5,1986.0,AN EVOLUTIONARY APPROACH TO NORMS,AMERICAN POLITICAL SCIENCE REVIEW,"{""AXELROD, R""}",80,NaN,1095,NaN,NaN,NaN,NaN,NaN,an evolutionary approach to norms
127927,WOS:000423016900007,WOS:000279784000002,2012,6,2010.0,Incentives and opportunism: from the carrot to...,PROCEEDINGS OF THE ROYAL SOCIETY B-BIOLOGICAL ...,"{""Hilbe, C""}",277,NaN,2427,10.1098/rspb.2010.0065,NaN,NaN,NaN,NaN,incentives and opportunism: from the carrot to...
127931,WOS:000423016900007,WOS:000281000100012,2012,10,2010.0,Choosing the Carrot or the Stick? Endogenous I...,REVIEW OF ECONOMIC STUDIES,"{""Sutter, M""}",77,NaN,1540,10.1111/j.1467-937X.2010.00608.x,NaN,NaN,NaN,NaN,choosing the carrot or the stick? endogenous i...


Create dictionaries to rename duplicate ids and to create a new column with the new ids

In [45]:
# for evolgath:
dictevol={}

for index,s in merged_evolgath.iterrows(): #loop over rows in our df
    if s['title_lower'] in dictevol: #if title already exists in our created dictionary
        if s['item_id_cited'] not in dictevol[s['title_lower']]: #and id_item_cited is still not present in our dictionary
            dictevol[s['title_lower']].append(s['item_id_cited']) #then append the id_item_cited to the existing title
    else: 
        dictevol[s['title_lower']] = [s['item_id_cited']] #if not, create new key title with item_id_cited as a value
        
len(dictevol) # 53648


# Check if the dictionary is correct
#print(list(dictevol.items())[:100])
#chunk_dictevol = {k: dictevol[k] for k in list(dictevol)[:20]}
#print(chunk_dictevol) #gives chunk of 20 titles

53649

In [46]:
# for coop:
dictcoop={}

for index,s in merged_coop.iterrows(): #loop over rows in our df
    if s['title_lower'] in dictcoop: #if title already exists in our created dictionary
        if s['item_id_cited'] not in dictcoop[s['title_lower']]: #and id_item_cited is still not present in our dictionary
            dictcoop[s['title_lower']].append(s['item_id_cited']) #then append the id_item_cited to the existing title
    else: 
        dictcoop[s['title_lower']] = [s['item_id_cited']] #if not, create new key title with item_id_cited as a value
        
len(dictcoop) # 28862

28862

Key-Value pairs with more than 1 entry need to be relabeled

In [47]:
# for evolgath:
dictdupevol = {k: dictevol[k] for k in dictevol if len(dictevol[k]) > 1} 

#print(dictdupevol)
len(dictdupevol) # 549

merged_evolgath.isna()
#print( sorted(dictdupevol.items(), key= lambda x: len(x[1]), reverse=True) ) #sort by number of values

del dictdupevol[np.nan] # nan needs the np in front!!
dictdupevol_sorted = sorted(dictdupevol.items(), key= lambda x: len(x[1]), reverse=True) # sort by number of values

#print(dictdupevol_sorted)

# for coop:
dictdupcoop = {k: dictcoop[k] for k in dictcoop if len(dictcoop[k]) > 1} 

#print(dictdupcoop)
len(dictdupcoop) # 549

merged_coop.isna()
#print(sorted(dictdupcoop.items(), key= lambda x: len(x[1]), reverse=True)) #sort by number of values

del dictdupcoop[np.nan] # nan needs the np in front!!
dictdupcoop_sorted = sorted(dictdupcoop.items(), key= lambda x: len(x[1]), reverse=True) # sort by number of values



In [48]:
# create df for evolgath multiple ids per lower case reference title
dfdupevol=pd.DataFrame.from_dict(dict(dictdupevol_sorted), orient='index') # df for better visibility
dfdupevol.to_csv('evolgath_titles_with_multi_ids.csv', encoding='utf-8')
dfdupevol #check only really for the bigger first one, the other ones are most probably the same

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,41,42,43
evolutionary game theory,000604466500067.7,000527769700042.48,000498881800052.4,000296007000001.31,000550684000036.9,000297819200015.9,000729391000001.9,000464880100077.10,000458654000246.16,000607329000002.10,...,000855235100018.14,000801530400001.48,000400033400006.35,000475440300003.14,000356864300125.4,000382389302072.20,000426422800029.33,000332858400012.11,000296042400022.24,000519214100002.34
stochastic evolutionary game dynamics,WOS:A1990EE29600005,000550684000036.11,000705615300001.39,000451665100001.52,000358468800002.95,000208819200007.6,000328609300009.7,000390640200001.104,000824314200001.16,000412164500007.60,...,None,None,None,None,None,None,None,None,None,None
population dynamics from game theory,000443369200005.34,000298615101113.14,000390640200001.114,000770156100001.47,000463926800012.52,000390611800074.56,000315578000022.49,000208819200003.51,000739761400001.34,000403140800143.23,...,None,None,None,None,None,None,None,None,None,None
random processes in genetics,000503281500014.23,000318996500002.85,000327688800029.19,000506466300026.56,000447274400007.35,000841295800002.84,000592161400001.27,000778570300002.41,000686555900007.49,000315428400001.13,...,None,None,None,None,None,None,None,None,None,None
"adaptive dynamics, a geometrical study of the consequences of nearly faithful reproduction",000375236800003.33,000386195400004.69,000316893000012.57,000679791300001.18,000391385200020.62,000364510400012.29,000367213300008.28,000334332400001.72,000308837900004.50,000359505200049.35,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
simultaneous iterative water-filling for gaussian frequency-selective interference channels,000320456000005.9,000348913400044.12,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
matrix exponential learning: distributed optimization in mimo systems,000320456000005.5,INSPEC:12963238,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
a tripartite game analysis on reverse logistics of express packaging under restricted control policies,000867145800001.47,000779618700002.16,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
risk assessment and prevention strategy of financial collateral for agricultural products logistics,000465038400021.19,000465038400021.15,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [49]:
# create df for coop multiple ids per lower case reference title
dfdupcoop=pd.DataFrame.from_dict(dict(dictdupcoop_sorted), orient='index') # df for better visibility
dfdupcoop.to_csv('coop_titles_with_multi_ids.csv', encoding='utf-8')
dfdupcoop #check only really for the bigger first one, the other ones are most probably the same

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
on the evolution of random graphs,000356184600016.23,000340692700017.8,000806160300004.83,WOS:A1960CAW3400027,000434313800002.14,000451107700005.2,000465207100086.8,000503281500014.11,000753670300020.63,000577084700012.10,...,000479157100002.6,000415298800001.6,000338430700016.12,000430383700002.16,000322342800078.9,000344384600001.14,000432778300015.9,000452093900046.10,000353311700013.5,000352337100001.10
innate social aptitudes of man: an approach from evolutionary genetics,000363763100010.27,000367094700004.44,000316584100005.19,000338236200001.40,000379268200018.19,000453088300002.27,000446774400001.53,000323940300003.37,000652670300004.23,000381633700014.59,...,000308748400020.18,000287080100009.32,000357813500025.15,000435181000001.26,000411981000026.34,None,None,None,None,None
random processes in genetics,000375067500027.23,000390305100001.41,000853048100001.44,000451107700007.18,000487964800011.28,000458224200009.32,000407873100009.30,000503281500014.23,000303035500004.32,000396964300003.44,...,000303844700006.28,000385471800005.29,000448438400011.42,000686555900007.49,000465480000014.31,None,None,None,None,None
public goods: a survey of experimental research,000392417600012.27,000700011100014.4,000308912600023.22,000308434100001.31,000354667000012.39,000360222300008.16,000307780400011.26,000353854200003.28,000354226700004.29,000361837200013.32,...,000381046000001.26,000400212800019.30,000314690900002.325,000393733300058.18,None,None,None,None,None,None
stochastic evolutionary game dynamics,WOS:A1990EE29600005,000540157600004.39,000389591600001.14,000302873700145.70,000308837900011.39,000451665100001.52,000574952100011.51,000426164700001.48,000752279100002.25,000435642400002.39,...,000305349100010.42,000307171100001.34,000686555900007.68,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
a social psychology of reputation,000349509100020.14,000371362600003.18,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
cooperative breeding in mammals,CCC:000237246800010,WOS:A1994MX95900006,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
marsupial sperm pairing: a case of 'sticky' green beards?,WOS:000174217000008,WOS:000174217000007,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
the evolution of cooperation in signed networks under the impact of structural balance,000694016500001.26,WOS:000446632700019,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


Replacing dictionary and adding new column with clean item_id_cited

In [50]:
merged_evolgath['item_id_clear'] = merged_evolgath['item_id_cited'] #clone item_id_cited column to later replace values only in there
merged_evolgath.head() # new column created
type(dictdupevol) # dict

dictreplace_evol ={v[0]: v[1:] for v in dictdupevol.values()} 
# ignore keys aka titles; take first id as key, remaining ids until last as values
# goal: {'BCI:BCI19522600023673': ["A", "B"]} 
dfdictreplace_evol=pd.DataFrame.from_dict(dict(dictdupevol_sorted), orient='index') # df for better visibility
dfdictreplace_evol
# rearranging dictionary to long format to avoid double looping through df and value lists

dictreplace_evol_alt = {i: k for k,v in dictreplace_evol.items() for i in v} # needs .items(), otherwise returns meta stuff
# k becomes first id
# v is remaining list of other ids

# same as:
#dictreplace_alt = {}
#for k,v in dictreplace.items():
#    for i in v:
#        dictreplace_alt[i]=k

dictreplace_evol_alt

{'000365283500228.7': 'WOS:000073092100002',
 '000309426500002.8': 'WOS:000073092100002',
 '000453704700031.4': 'WOS:000223799100054',
 'WOS:000242408800016': '000391330900109.20',
 '000354867200038.19': 'WOS:000242624600030',
 'WOS:A1992JV77700063': '000391330900109.9',
 '000309983400011.2': 'WOS:000220697200041',
 '000378641000001.15': 'WOS:A1979HZ08500018',
 '000432507200005.21': 'WOS:A1979HZ08500018',
 '000326414400011.61': 'WOS:A1979HZ08500018',
 '000458114803053.24': 'WOS:A1979HZ08500018',
 '000307026700061.10': 'WOS:A1979HZ08500018',
 '000296042400022.23': 'WOS:A1979HZ08500018',
 '000301755700018.36': 'WOS:A1978FK24700004',
 '000301970600014.15': 'WOS:A1978FK24700004',
 '000492952400013.15': 'WOS:A1978FK24700004',
 '000501728600076.23': 'WOS:A1978FK24700004',
 '000311051000007.11': 'WOS:A19647207B00008',
 '000334332400001.39': '000632837000003.16',
 '000388830700022.19': '000632837000003.16',
 '000309990600011.54': 'WOS:000268616500013',
 '000378833300017.28': 'WOS:0002849967000

In [51]:
merged_evolgath2 = merged_evolgath.replace({'item_id_clear':dictreplace_evol_alt})
merged_evolgath.compare(merged_evolgath2)

item_id_clear                     
                       self                other
170     WOS:A1992JV77700063    000391330900109.9
236     WOS:A1992JV77700063    000391330900109.9
798     WOS:A1992JV77700063    000391330900109.9
1282    WOS:A1992JV77700063    000391330900109.9
1501    WOS:A1992JV77700063    000391330900109.9
...                     ...                  ...
142663   000401121200005.17   000309943600046.14
142691  WOS:000083578500007  WOS:000086093700078
142703   000423620300001.28   000402076700034.33
142769  WOS:000606811900021   000660584400009.30
142772  WOS:A1985AJH8100011   000313365200004.20

[3051 rows x 2 columns]

Manually check most frequent titles: "evolutionary game theory"

In [52]:
# check titles with most ids, take care of indexing
# merged_evolgath2.loc[merged_evolgath2['title_lower'] == 'evolutionary game theory'] # uncomment to see all rows

# manual check for distinct articles with title "evolutionary game theory"
weibull = [2809,19703,21738,34994,48101,56850, 70501, 86011, 88725, 89908, 104318, 108880, 123500] # weibull
merged_evolgath2.loc[weibull,'item_id_clear'] = '000527769700042.48'

easley = [7354,71273]
merged_evolgath2.loc[easley, 'item_id_clear'] = '000498881800052.4' # easley, not easeley, can't find publication

sandholm = [408, 8738, 9257, 19512, 52016, 53875, 60557, 62013,
            73592, 79577, 84930, 97232, 103736, 105023, 115969, 120211, 133796, 138595, 139319, 141443] # sandholm
merged_evolgath2.loc[sandholm,'item_id_clear'] = '000604466500067.7'

alexander = [12024, 29023, 32574, 37342, 44444, 57100, 76397, 106633, 106633, 108286, 114958, 128102] # J. McKenzie Alexander
merged_evolgath2.loc[alexander,'item_id_clear'] = '000297819200015.9'

cressman = [27274, 62336, 92974, 95029]
merged_evolgath2.loc[cressman, 'item_id_clear'] = '000607329000002.10' # cressman

merged_evolgath2.loc[37946, 'item_id_clear'] = '000525401500057.17' # smith

sigmund = [46560, 63312, 135602]
merged_evolgath2.loc[sigmund, 'item_id_clear'] = '000457952400133.21' # sigmund

merged_evolgath2.loc[50085, 'item_id_clear'] = '000441998400054.33' # vincent

merged_evolgath2.loc[85323, 'item_id_clear'] = '000540458900001.23' # marsh

mcnamara = [62561, 121304, 134471]
merged_evolgath2.loc[mcnamara, 'item_id_clear'] = '000377890900002.52' # marsh

print(merged_evolgath.item_id_clear.nunique())
print(merged_evolgath2.item_id_clear.nunique())

73306
71405


Manually check most frequent titles: "stochastic evolutionary game dynamics"

In [53]:
# check titles with most ids, take care of indexing
#pd.set_option('display.max_rows', None)
#merged_evolgath2.loc[merged_evolgath2['title_lower'] == 'stochastic evolutionary game dynamics'] # uncomment to see all rows

# foster will keep given clear id

wallace = [9259, 56675, 73594, 127761]
merged_evolgath2.loc[wallace, 'item_id_clear'] = '000550684000036.11' # wallace

traulsen = [9911, 10864, 11060, 13370, 16064, 18399, 21053, 21296, 26125, 30109, 36098, 43663, 47440, 48532, 50225, 
            58148, 65067, 75206, 82311, 103020, 113254, 118672, 120963, 121367, 127141, 129041, 132124]
merged_evolgath2.loc[traulsen, 'item_id_clear'] = '000705615300001.39' # traulsen

hauert = [34608, 103020]
merged_evolgath2.loc[hauert, 'item_id_clear'] = '000453111000009.79' # hauert

print(merged_evolgath.item_id_clear.nunique())
print(merged_evolgath2.item_id_clear.nunique()) # three ids added to last step


73306
71408


Manually check most frequent titles: "population dynamics from game theory"

In [54]:
# check titles with most ids, take care of indexing
#merged_evolgath2.loc[merged_evolgath2['title_lower'] == 'population dynamics from game theory'] # uncomment to see all rows

# no steps needed, all entries from Zeeman.

Manually check most frequent titles: "random processes in genetics"

In [55]:
# check titles with most ids, take care of indexing
#merged_evolgath2.loc[merged_evolgath2['title_lower'] == 'random processes in genetics'] # uncomment to see all rows

# no steps needed, all entries from Moran.

Manually check for most frequent titles: "adaptive dynamics, a geometrical study of the consequences of nearly faithful reproduction"

In [56]:
# check titles with most ids, take care of indexing
#merged_evolgath2.loc[merged_evolgath2['title_lower'] == 'adaptive dynamics, a geometrical study of the consequences of nearly faithful reproduction'] # uncomment to see all rows

# no steps needed, all entries from Metz.

Drop columns without sufficient information

In [57]:
# drop useless columns that contain mostly nans
merged_evolgath2 = merged_evolgath2.drop(columns=['scopus_ref_issue', 
                        'wos_ref_article_number', 
                        'scopus_ref_text', 
                        'scopus_ref_fulltext', 
                        'wos_citation_context' ])
#merged_evolgath2.head()

Apply filter of minimum 2 citations

In [58]:
len(merged_evolgath2)

142910

In [63]:
series_evolgath2 = merged_evolgath2.groupby("item_id_clear")['item_id_citing'].nunique() #group the cited id by the citing id
#print(series_evolgath) #Length: 73306

# for evolgath:
# cited items with more than one citing item
ab1_evol2 = series_evolgath2.loc[series_evolgath2.values > 1]

# show positions of the cited items with more than one citing item
merged_evolgath2 = merged_evolgath2.loc[merged_evolgath2.item_id_clear.isin(ab1_evol2.index)].reset_index(drop=True)

len(merged_evolgath2) # 68658

68658

In [64]:
merged_evolgath2.head

<bound method NDFrame.head of             item_id_citing        item_id_cited  citing_pubyear  ref_seq_nr  \
0      WOS:000508289100054  WOS:000223113500001            2017           1   
1      WOS:000508289100054  WOS:000073092100002            2017           3   
2      WOS:000274798200004  WOS:A1989U899000001            2009           5   
3      WOS:000274798200004  WOS:000188753800045            2009           7   
4      WOS:000274798200004  WOS:000223799100054            2009           8   
...                    ...                  ...             ...         ...   
68653  WOS:000076056600001  WOS:A1992JM57700006            1998          68   
68654  WOS:000076056600001  WOS:A1989CC40500006            1998          71   
68655  WOS:000076056600001  WOS:A1993MD68700030            1998          72   
68656  WOS:000076056600001  WOS:A1993KH52900002            1998          74   
68657  WOS:000076056600001  WOS:A1992JM57700009            1998          75   

       ref_pubyear   

In [60]:
merged_evolgath2.to_csv('merged_evolgath_clean.csv')

Equivalent procedure for merged_coop: Replacing dictionary and adding new column with clean item_id_cited

In [65]:
merged_coop['item_id_clear'] = merged_coop['item_id_cited'] #clone item_id_cited column to later replace values only in there
merged_coop.head() # new column created
type(dictdupcoop) # dict

dictreplace_coop ={v[0]: v[1:] for v in dictdupcoop.values()} 
# ignore keys aka titles; take first id as key, remaining ids until last as values
# goal: {'BCI:BCI19522600023673': ["A", "B"]} 
dfdictreplace_coop=pd.DataFrame.from_dict(dict(dictdupcoop_sorted), orient='index') # df for better visibility
dfdictreplace_coop
# rearranging dictionary to long format to avoid double looping through df and value lists

dictreplace_coop_alt = {i: k for k,v in dictreplace_coop.items() for i in v} # needs .items(), otherwise returns meta stuff
# k becomes first id
# v is remaining list of other ids

# same as:
#dictreplace_alt = {}
#for k,v in dictreplace.items():
#    for i in v:
#        dictreplace_alt[i]=k

dictreplace_coop_alt

{'000459131600002.19': 'WOS:000533938000001',
 '000426537600014.28': 'WOS:000533938000001',
 '000382423700011.10': 'WOS:000074150100051',
 '000295049102022.28': 'WOS:000074150100051',
 '000430383700002.48': 'WOS:000345920800037',
 '000369643200002.15': 'WOS:000345920800037',
 '000365312800010.25': '000803916300001.47',
 '000404850000001.27': '000803916300001.47',
 '000380514500293.11': '000803916300001.47',
 '000308785100044.12': 'WOS:000290584400002',
 '000417519600058.30': 'WOS:000231503600068',
 '000441492100019.13': 'WOS:000259263700029',
 '000391002500069.6': 'WOS:000259263700029',
 '000323458700008.152': 'WOS:000288247800004',
 '000299471800038.33': 'WOS:000297463100026',
 '000376857800001.3': 'WOS:000284769200004',
 'WOS:A1981LY39000024': 'WOS:A1981LG07300011',
 'WOS:000222161000001': 'WOS:A1981LG07300011',
 '000404700100013.2': 'WOS:A1981LG07300011',
 '000446632700019.2': 'WOS:A1981LG07300011',
 'WOS:A1997XA45300005': 'WOS:A1981LG07300011',
 '000306567500009.24': 'WOS:000248365

In [66]:
merged_coop2 = merged_coop.replace({'item_id_clear':dictreplace_coop_alt})
#merged_coop.compare(merged_coop2)

In [67]:
# check titles with most ids, take care of indexing
merged_coop2.loc[merged_coop2['title_lower'] == 'evolutionary game theory'] # uncomment to see all rows

,item_id_citing,item_id_cited,citing_pubyear,ref_seq_nr,ref_pubyear,ref_item_title,ref_source_title,ref_authors,ref_volume,scopus_ref_issue,ref_pages,ref_doi,wos_ref_article_number,scopus_ref_text,scopus_ref_fulltext,wos_citation_context,title_lower,item_id_clear
6303,WOS:000865985500094,000865985500094.3,2020,31,1000.0,Evolutionary Game Theory,EVOLUTIONARY GAME TH,"{""Weibull, Jorgen W.""}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,evolutionary game theory,000865985500094.3
36218,WOS:000426422800009,000426422800009.21,2018,25,2009.0,Evolutionary game theory,Encyclopedia of complexity and systems science,"{""Sandholm, WH""}",NaN,NaN,3176,NaN,NaN,NaN,NaN,NaN,evolutionary game theory,000865985500094.3
37962,WOS:000527769700042,000527769700042.48,2020,15,1995.0,Evolutionary game theory,Current Biology,"{""Weibull, J. W.""}",9,NaN,503,NaN,NaN,NaN,NaN,NaN,evolutionary game theory,000865985500094.3
54546,WOS:000441492100067,000441492100067.28,2018,10,2009.0,Evolutionary game theory,Encyclopedia of Complexity and Systems Science,"{""Sandholm, W.H.""}",NaN,NaN,3176,NaN,NaN,NaN,NaN,NaN,evolutionary game theory,000865985500094.3
76352,WOS:000798992800014,000798992800014.12,2022,33,2020.0,Evolutionary game theory,Complex Social and Behavioral Systems: Game Th...,"{""Sandholm, WH""}",NaN,NaN,573,NaN,NaN,NaN,NaN,NaN,evolutionary game theory,000865985500094.3
94457,WOS:000332858400012,000332858400012.11,2014,19,2010.0,Evolutionary game theory,"Social Behaviour Genes, Ecology and Evolution","{""McNamara, J""}",NaN,NaN,109,NaN,NaN,NaN,NaN,NaN,evolutionary game theory,000865985500094.3
94783,WOS:000305341000022,000305341000022.23,2012,13,2010.0,Evolutionary Game Theory,"Social Behaviour. Genes, Ecology and Evolution","{""McNamara, JM""}",NaN,NaN,109,NaN,NaN,NaN,NaN,NaN,evolutionary game theory,000865985500094.3
106340,WOS:000474545500066,000474545500066.43,2019,57,2009.0,Evolutionary game theory,Encyclopedia of Complexity and Systems Science,"{""Sandholm, W.H.""}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,evolutionary game theory,000865985500094.3
111784,WOS:000441492100067,000441492100067.28,2018,10,2009.0,Evolutionary game theory,Encyclopedia of Complexity and Systems Science,"{""Sandholm, W.H.""}",NaN,NaN,3176,NaN,NaN,NaN,NaN,NaN,evolutionary game theory,000865985500094.3


Manually check most frequent titles: "on the evolution of random graphs"

In [71]:
# check titles with most ids, take care of indexing
#merged_coop2.loc[merged_coop2['title_lower'] == 'on the evolution of random graphs'] # uncomment to see all rows

# manual check for distinct articles with title "on the evolution of random graphs"

# Erdos keeps given clear id

renyi = [45870,120488]
merged_coop2.loc[renyi, 'item_id_clear'] = '000577084700012.10' # renyi

#print(merged_coop.item_id_clear.nunique())
#print(merged_coop2.item_id_clear.nunique())

Manually check most frequent titles: "innate social aptitudes of man: an approach from evolutionary genetics"

In [74]:
# check titles with most ids, take care of indexing
#merged_coop2.loc[merged_coop2['title_lower'] == 'innate social aptitudes of man: an approach from evolutionary genetics'] # uncomment to see all rows

# different authors result from
# Hamilton, W.D. (1975), Innate social aptitudes of man: an approach from evolutionary genetics, in R. Fox (ed.), Biosocial Anthropology, Malaby Press, London, pp. 133-153.

#print(merged_coop.item_id_clear.nunique())
#print(merged_coop2.item_id_clear.nunique())

Manually check most frequent titles: "random processes in genetics"

In [76]:
# check titles with most ids, take care of indexing
#merged_coop2.loc[merged_coop2['title_lower'] == 'random processes in genetics'] # uncomment to see all rows

# no steps needed, all entries from Moran.

Manually check most frequent titles: "public goods: a survey of experimental research

In [78]:
# check titles with most ids, take care of indexing
#merged_coop2.loc[merged_coop2['title_lower'] == 'public goods: a survey of experimental research'] # uncomment to see all rows

# no steps needed, all entries from Ledyard.

Manually check most frequent titles: "stochastic evolutionary game dynamics"

In [80]:
# check titles with most ids, take care of indexing
merged_coop2.loc[merged_coop2['title_lower'] == 'stochastic evolutionary game dynamics'] # uncomment to see all rows

traulsen = [3934, 13644, 13946, 20423, 47024, 47816, 48839, 49370, 56686, 70024, 71849, 76154, 77020, 100657, 102169, 102772]
merged_coop2.loc[traulsen, 'item_id_clear'] = '000540157600004.39' # traulsen


Dropp columns without sufficient information

In [81]:
# drop useless columns that contain mostly nans
merged_coop2 = merged_coop2.drop(columns=['scopus_ref_issue', 
                        'wos_ref_article_number', 
                        'scopus_ref_text', 
                        'scopus_ref_fulltext', 
                        'wos_citation_context' ])
#merged_coop2.head()

Apply filter of minimum 2 citations

In [82]:
len(merged_coop2)

127934

In [83]:
series_coop2 = merged_coop2.groupby("item_id_clear")['item_id_citing'].nunique()
#print(series_coop) # 42797

# for coop
ab1_coop2 = series_coop2.loc[series_coop2.values > 1]
merged_coop2 = merged_coop2.loc[merged_coop2.item_id_clear.isin(ab1_coop2.index)].reset_index(drop=True)
merged_coop2.head

<bound method NDFrame.head of             item_id_citing        item_id_cited  citing_pubyear  ref_seq_nr  \
0      WOS:000803916300001  WOS:000188753800045            2022           8   
1      WOS:000803916300001  WOS:000241921500011            2022           6   
2      WOS:000803916300001  WOS:000301775800001            2022           2   
3      WOS:000803916300001  WOS:000369085100062            2022           3   
4      WOS:000803916300001  WOS:000322521900002            2022           4   
...                    ...                  ...             ...         ...   
89440  WOS:000423016900007  WOS:000232993500002            2012           4   
89441  WOS:000423016900007  WOS:A1986F589100001            2012           5   
89442  WOS:000423016900007  WOS:000279784000002            2012           6   
89443  WOS:000423016900007  WOS:000281000100012            2012          10   
89444  WOS:000423016900007  WOS:000444387200015            2012          12   

       ref_pubyear   

In [84]:
len(merged_coop2)

89445

In [85]:
merged_coop2.to_csv('merged_coop_clean.csv')